LSE Data Science Institute | DS105W (2023/24) | Week 07

# 🗓️ Week 07: Putting everything together

Theme: Collecting Data

**DATE:** 29 February 2024

**AUTHOR:** Dr [Jon Cardoso-Silva](https://jonjoncardoso.github.io)

-----



**USEFUL LINKS:**

- [UK general elections](https://en.wikipedia.org/wiki/List_of_United_Kingdom_general_elections)

⚙️ Setup

In [2]:
import io
import requests

import pandas as pd
import numpy as np

from scrapy.selector import Selector
from tqdm.notebook import tqdm

# Part 1: Pages about UK general elections

In [3]:
base_url = 'https://en.wikipedia.org/wiki/List_of_United_Kingdom_general_elections'

# Get the page using requests and Scrapy
response = requests.get(base_url)
sel = Selector(text=response.text)

response.status_code

200

**Now I will get the list of pages:**

In [21]:
## Convert the above to XPATH

all_links = \
(
    sel.xpath("(//td[contains(@class, 'navbox-list')])[1]")
       .css('div > ul > li ::attr(href)')
       .extract()
)

In [22]:
all_links

['/wiki/First_Parliament_of_the_United_Kingdom',
 '/wiki/1802_United_Kingdom_general_election',
 '/wiki/1806_United_Kingdom_general_election',
 '/wiki/1807_United_Kingdom_general_election',
 '/wiki/1812_United_Kingdom_general_election',
 '/wiki/1818_United_Kingdom_general_election',
 '/wiki/1820_United_Kingdom_general_election',
 '/wiki/1826_United_Kingdom_general_election',
 '/wiki/1830_United_Kingdom_general_election',
 '/wiki/1831_United_Kingdom_general_election',
 '/wiki/1832_United_Kingdom_general_election',
 '/wiki/1835_United_Kingdom_general_election',
 '/wiki/1837_United_Kingdom_general_election',
 '/wiki/1841_United_Kingdom_general_election',
 '/wiki/1847_United_Kingdom_general_election',
 '/wiki/1852_United_Kingdom_general_election',
 '/wiki/1857_United_Kingdom_general_election',
 '/wiki/1859_United_Kingdom_general_election',
 '/wiki/1865_United_Kingdom_general_election',
 '/wiki/1868_United_Kingdom_general_election',
 '/wiki/1874_United_Kingdom_general_election',
 '/wiki/188

## Intermediate step to get the page with breakdown of results

In [25]:
prefix = 'https://en.wikipedia.org'
current_page = '/wiki/2019_United_Kingdom_general_election'

# Get the page using requests and Scrapy
response = requests.get(prefix + current_page)

# Select the <a> that contains the text 'Results of the ' in its title
results_link = \
(
    Selector(text=response.text)
        .xpath("//a[contains(@title, 'Results of the 2019')]")
        .css('::attr(href)')
        .extract_first()
)

results_link

'/wiki/Results_of_the_2019_United_Kingdom_general_election'

In [33]:
current_page = results_link

# Get the page using requests and Scrapy
response = requests.get(prefix + current_page)
sel = Selector(text=response.text)

# Convert the CSS selector above to XPATH

df = pd.read_html(sel.xpath('(//table[contains(@class, "wikitable")])[1]').extract_first())[0]

C:\Users\Jon\AppData\Local\Temp\ipykernel_12964\2583135114.py:9: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(sel.xpath('(//table[contains(@class, "wikitable")])[1]').extract_first())[0]


**Filter out the lines I don't care about:**

In [ ]:
valid_rows = df['Constituency'] != 'Total for all constituencies'
valid_rows = np.array(valid_rows.values).flatten()

df[valid_rows]

**Without pd.read_html**

In [47]:
columns = sel.xpath('(//table[contains(@class, "wikitable")])[1]').xpath('./tbody/tr[th and not(contains(@class, "sortbottom"))]').extract()

In [48]:
rows = sel.xpath('(//table[contains(@class, "wikitable")])[1]').xpath('./tbody/tr[not(th)]')

In [54]:
rows[0].xpath('./td/text()').extract()

['\n',
 '\n',
 '\n',
 '27,980\n',
 '58.4%\n',
 '16,698\n',
 '66.0%\n',
 '27,980\n',
 '11,282\n',
 '6,920\n',
 '1,750\n',
 '\n',
 '\n',
 '47,932\n']